# Importing Necessary Libraries

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
import re
nltk.download("popular")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

# Import Dataset

In [65]:
!wget -O train.csv https://github.com/koiralasandesh/fake_news/raw/master/train.csv
data_file="train.csv"
df=pd.read_csv(data_file)

--2021-02-15 18:58:12--  https://github.com/koiralasandesh/fake_news/raw/master/train.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/koiralasandesh/fake_news/master/train.csv [following]
--2021-02-15 18:58:12--  https://raw.githubusercontent.com/koiralasandesh/fake_news/master/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98628550 (94M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  94.06M   126MB/s    in 0.7s    

2021-02-15 18:58:13 (126 MB/s) - ‘train.csv’ saved [98628550/98628550]



# Preprocessing

Now, lets view the dataset info

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


Lets view some data from the top and bottom of our dataset

In [67]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


We now see the number of null values in the columns 'title' and 'text'.

In [68]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Now we drop the objects with null values in title and text.
We also see the number of null values in each column

In [69]:
df=df.dropna(subset=['title','text'])
df.isnull().sum()

id           0
title        0
author    1918
text         0
label        0
dtype: int64

In [70]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Now we need to remove duplicate from the dataset.

In [71]:
df=df.drop_duplicates(subset=['text'])
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Appending the "title" and "text" into same field.

In [72]:
df['title_text'] = df['title']+df['text']
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,title,author,text,label,title_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredWhy the Truth...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


Dropping "id","title","author" and "text" column.

In [73]:
df=df.drop(['id','title','author','text'],axis=1)
df

,label,title_text
0,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,1,Why the Truth Might Get You FiredWhy the Truth...
3,1,15 Civilians Killed In Single US Airstrike Hav...
4,1,Iranian woman jailed for fictional unpublished...
...,...,...
20795,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,0,Macy’s Is Said to Receive Takeover Approach by...
20798,1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [74]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

lst_stopwords = nltk.corpus.stopwords.words("english")

In [75]:
df['clean_text']=df["title_text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
df

,label,title_text,clean_text
0,1,House Dem Aide: We Didn’t Even See Comey’s Let...,house dem aide didnt even see comeys letter ja...
1,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",flynn hillary clinton big woman campus breitba...
2,1,Why the Truth Might Get You FiredWhy the Truth...,truth might get firedwhy truth might get fired...
3,1,15 Civilians Killed In Single US Airstrike Hav...,15 civilian killed single u airstrike identifi...
4,1,Iranian woman jailed for fictional unpublished...,iranian woman jailed fictional unpublished sto...
...,...,...,...
20795,0,Rapper T.I.: Trump a ’Poster Child For White S...,rapper ti trump poster child white supremacyra...
20796,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",nfl playoff schedule matchup odds new york tim...
20797,0,Macy’s Is Said to Receive Takeover Approach by...,macys said receive takeover approach hudson ba...
20798,1,"NATO, Russia To Hold Parallel Exercises In Bal...",nato russia hold parallel exercise balkansnato...


In [76]:
xfeed=df['clean_text']
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
cv= TfidfVectorizer()
x=cv.fit_transform(xfeed)

In [77]:
y= df['label']

# Model Analysis

We split the data into train and test dataset:

In [78]:
train_feature, test_feature, train_class, test_class = train_test_split(
    x, y,stratify=y)

## KNN Classifier

In [79]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(train_feature, train_class)
print("Test set predictions:\n{}".format(knn.predict(test_feature)))
print("Test set accuracy: {:.2f}".format(knn.score(test_feature, test_class)))

Test set predictions:
[1 1 0 ... 1 0 0]
Test set accuracy: 0.82


## Decision Tree

In [80]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(train_feature, train_class)
print("Training set score: {:.3f}".format(tree.score(train_feature, train_class)))
print("Test set score: {:.3f}".format(tree.score(test_feature, test_class)))

Training set score: 1.000
Test set score: 0.914


##Support Vector Machine

In [81]:
linearsvm = LinearSVC().fit(train_feature, train_class)
print("Test set score: {:.3f}".format(linearsvm.score(test_feature, test_class)))

Test set score: 0.958


In [82]:
linearsvm=LinearSVC().fit(x,y)

# prediction Test

In [83]:
text1='''
While President Trump continues to baselessly allege widespread election fraud, the political world is starting to move on from his presidency.

Key states have certified their election results.

Republican election officials have pushed back against the president's rhetoric, which they say led to threats against election workers.

"Mr. President, you have not condemned these actions or this language," Gabriel Sterling, the voting system implementation manager in the Georgia secretary of state's office, said Tuesday. "Senators, you have not condemned this language or these actions. This has to stop. We need you to step up. And if you take a position of leadership, show some."

'''


text=utils_preprocess_text(text1, flg_stemm=False, flg_lemm=True, 
           lst_stopwords=lst_stopwords)

In [84]:
def predictText(text):
  quest=cv.transform([text])
  x=linearsvm.predict(quest)
  if x[0]==0:
    print("This text is machine generated!")
  elif x[1]==1:
    print("This text was written by humans!")
  else:
    print("There was an error, we could not make any prediction.")


# Parameter Tuning

## Decision Tree

In [85]:
# params={'max_depth':range(1,500)}
# tree = GridSearchCV(DecisionTreeClassifier(),params,verbose = 1,cv=5,n_jobs=10)
# tree_results=tree.fit(x,y)

In [86]:
#tree_results.best_score_

## KNN


In [87]:
# params= {'n_neighbors': range(1,5)}
# gs = GridSearchCV(KNeighborsClassifier(),params,verbose = 1,cv=10,n_jobs=10)
# gs_results= gs.fit(x,y)

In [88]:
# gs_results.best_score_

## SVM


In [89]:
# from sklearn.svm import SVC
# params ={'kernel':['rbf','poly','sigmoid'],'max_iter':[2000,3000,4000]}
# grid_svc=GridSearchCV(SVC(),params,verbose =1,cv=10,n_jobs=-1)
# grid_svc_results= grid_svc.fit(nba_feature,nba_class)

In [90]:
# grid_svc_results.best_score_

## NB


In [91]:
# import matplotlib.pyplot as plt
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# scores = cross_val_score(classifier,x,y,n_jobs=-1,cv = 10)
# print(scores)
# plt.figure(figsize=(15,10))
# plt.plot(scores)
# plt.title("Cross Validation Scores per run")
# plt.xlabel("CV Run")
# plt.ylabel("Score") 
# plt.show()
# print(scores.mean())
# train_feature, test_feature, train_class, test_class = train_test_split(
#     x.toarray(),y, stratify=y)
# nb = GaussianNB().fit(train_feature, train_class)
# print("Test set score: {:.3f}".format(nb.score(test_feature, test_class)))

## Random Forest

In [92]:
# from sklearn.ensemble import RandomForestClassifier


#Final Prediction

In [93]:
text = '''

Students across the nation are dealing with pandemic hardships, online classes and economic insecurity. In November, The Shorthorn reported that students felt drained, unmotivated and burned out.

As the semester winds down, we want to encourage students to push through and finish strong. This has been a rough time for everyone, including us. But we believe in you and your ability to succeed in the face of adversity.

According to previous reporting by The Shorthorn, some students decided to skip the fall semester because of economic uncertainty and fears related to the virus. Those who stayed dealt with these hardships on top of a difficult transition to remote learning.

Students have voiced concerns about online classes not translating well in some majors. Others have spoken about the routine struggle to get internet connection. This is the first time many professors are teaching online courses, and their students feel the effect. 

'''
predictText(text)

This text is machine generated!
